<a href="https://colab.research.google.com/github/ChiliJoe/genai-intro/blob/main/ChatGPT_Function_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an expansion of tutorial from https://youtu.be/0-zlUy7VUjg using real-world API from openweathermap.org.

Let's use the openweathermap current weather API that requires geo code as input.

In [1]:
%%capture
!pip install openai google-search-results

In [2]:
%%capture
# Set serpapi API key
%env SERPAPI_API_KEY=

# Set openweathermap.org API key
%env OWM_API_KEY=

# Set OpenAI API key
%env OPENAI_API_KEY=

In [3]:
%%capture
!rm -rf helper
!git clone https://github.com/ChiliJoe/genai-intro.git helper

In [4]:
from helper.function_calling_demo import function_list

In [5]:
function_list

{'do_search': {'function': <function helper.function_calling_demo.do_search(search_term)>,
  'description': {'name': 'do_search',
   'description': 'Search google for current information',
   'parameters': {'type': 'object',
    'properties': {'search_term': {'type': 'string',
      'description': 'The term to search for in the web'}},
    'required': ['search_term']}}},
 'get_current_weather': {'function': <function helper.function_calling_demo.get_current_weather(lat, lon, unit)>,
  'description': {'name': 'get_current_weather',
   'description': 'Get the current weather in a given geo code',
   'parameters': {'type': 'object',
    'properties': {'lat': {'type': 'number', 'description': 'latitude'},
     'lon': {'type': 'number', 'description': 'latitude'},
     'unit': {'type': 'string',
      'description': 'The temperature unit to use. Infer this from the location.',
      'enum': ['standard', 'metric', 'imperial']}},
    'required': ['lat', 'lon', 'unit']}}},
 'get_location_data'

# Vanilla OpenAI

In [6]:
from IPython.display import display, Markdown
from helper.demolib import Color

import openai
import os
import json

openai.api_key  = os.getenv('OPENAI_API_KEY')

class Chat:
  def __init__(self, system_message, function_list):
    self.messages = [{"role": "system", "content": system_message}]
    self.function_list = function_list
    self.function_descriptions = [f["description"] for f in function_list.values()]

  def ask(self, role_content, role="user", function_name=None):
    if role == "function":
      self.messages.append({"role": role, "name": function_name, "content": role_content})
    else:
      self.messages.append({"role": role, "content": role_content})

    res = openai.ChatCompletion.create(
        model="gpt-4",
        messages=self.messages,
        functions=self.function_descriptions,
        function_call="auto"
    )
    ai_message = res.choices[0].message
    self.messages.append(ai_message)
    # print(ai_message)

    # process function call
    if ai_message["content"] is None and ai_message["function_call"]:
      target_function_name = ai_message["function_call"]["name"]
      target_function = self.function_list[target_function_name]["function"]
      func_params = json.loads(ai_message['function_call']['arguments'])

      print(Color.cyan(f"Invoking {target_function_name} with parameters {func_params}"))
      function_response = target_function(**func_params)
      print(Color.white("Function response: "), end="")
      print(Color.magenta(f"{function_response}"))

      self.ask(role_content=json.dumps(function_response),
               role="function",
               function_name=target_function_name
               )

    elif ai_message["content"]:
      print(Color.green("===== Final response: ====="))
      display(Markdown(ai_message["content"]))

In [7]:
sysmessage="""
You are a helpful AI chatbot that responds in conversational manner. Search the web for information that may need to be kept up-to-date.
"""
chat = Chat(sysmessage, function_list)

In [8]:
chat.ask("Who was the president of the Philippines in 2022? Where was he born?")

Invoking do_search with parameters {'search_term': 'President of the Philippines 2022'}
Function response: [{'position': 1, 'title': 'Presidency of Bongbong Marcos', 'snippet': 'The presidency of Bongbong Marcos began at noon on June 30, 2022, following his inauguration as the 17th president of the Philippines, succeeding Rodrigo ...', 'languages': 'English', 'regions': 'United States'}, {'position': 2, 'title': 'A Marcos returns to power in the Philippines', 'snippet': 'As Ferdinand Marcos Jr., son of the former dictator, is elected president in the Philippines, Mely Caballero-Anthony describes what the ...', 'languages': 'English', 'regions': 'United States'}, {'position': 3, 'title': 'Ferdinand Marcos Jr sworn in as Philippines president', 'snippet': 'Marcos Jr takes Philippines top job, 36 years after his father was toppled and forced into exile in a popular uprising.', 'languages': 'English', 'regions': 'United States'}, {'position': 4, 'title': 'Ferdinand Marcos Jr. is sworn in a

The president of the Philippines in 2022 was Bongbong Marcos. He was born in the town of Batac, Ilocos Norte, Philippines.

In [9]:
chat.ask("What is the weather like where he was born?")

Invoking get_location_data with parameters {'city': 'Batac', 'country': 'Philippines'}
Function response: [{'name': 'Batac', 'local_names': {'ja': 'バタック'}, 'lat': 18.0555035, 'lon': 120.5645243, 'country': 'PH', 'state': 'Ilocos Norte'}]
Invoking get_current_weather with parameters {'lat': 18.0555035, 'lon': 120.5645243, 'unit': 'metric'}
Function response: {'coord': {'lon': 120.5645, 'lat': 18.0555}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 26.13, 'feels_like': 26.13, 'temp_min': 26.13, 'temp_max': 26.13, 'pressure': 1003, 'humidity': 88, 'sea_level': 1003, 'grnd_level': 1001}, 'visibility': 10000, 'wind': {'speed': 6.14, 'deg': 182, 'gust': 9.07}, 'clouds': {'all': 100}, 'dt': 1693762284, 'sys': {'country': 'PH', 'sunrise': 1693777439, 'sunset': 1693822230}, 'timezone': 28800, 'id': 1726339, 'name': 'Batac City', 'cod': 200}
===== Final response: =====


The current weather in Batac, Ilocos Norte, Philippines is overcast with a temperature of 26.13°C. The humidity is at 88% with wind speed at 6.14 m/s coming from the south.